<a href="https://colab.research.google.com/github/JakubChmielewskiRepo/wstep-do-uczenia-maszynowego-175ic/blob/master/lab10/stackAbuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install aiohttp
import time
import nest_asyncio
import requests
import json
from urllib import request
import multiprocessing
import asyncio
import aiohttp
from concurrent.futures import ThreadPoolExecutor

     |████████████████████████████████| 1.3MB 6.2MB/s 
     |████████████████████████████████| 143kB 20.0MB/s 
     |████████████████████████████████| 296kB 18.0MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=06348449b5049fbb897dc6692c7e919d11d75ab8e1bf97f2d978d28c048fc16b
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


In [17]:
#Synchronous
def download_images():
    response = requests.get("https://picsum.photos/v2/list")
    if response.status_code != 200:
        raise AttributeError('GET /tasks/ {}'.format(response.status_code))
    data = json.loads(response.text)[:10]

    pictures=[]
    for s in data:
        pictures.append(s['download_url']+".jpg")
    return pictures

def saveImages(link):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    request.urlretrieve(link, "images/{}.{}".format(filename, fileformat))

def main():
    images = download_images()
    for image in images:
        saveImages(image)

start_time = time.time()
main()
duration_synch = time.time() - start_time
print(f"Time taken: {duration_synch}")

Time taken: 9.95365858078003


In [18]:
def process_images_threading():
    images = download_images()
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(saveImages,images)

start_time = time.time()
process_images_threading()
duration_multithreading = time.time() - start_time
print(f"Time taken: {duration_multithreading}")

Time taken: 2.4338181018829346


In [19]:
async def download_images_asyncio(link, session):
    filename = link.split('/')[6].split('.')[0]
    fileformat = link.split('/')[6].split('.')[1]
    async with session.get(link) as response:
        with open("picture/{}.{}".format(filename, fileformat), 'wb') as fd:
            async for data in response.content.iter_chunked(1024):
                fd.write(data)

async def main_asyncio():
    images = download_images()

    async with aiohttp.ClientSession() as session:
        tasks=[download_images_asyncio(image,session)for image in images]
        return await asyncio.gather(*tasks)

start_time = time.time()
nest_asyncio.apply()
asyncio.run(main_asyncio())
duration_asyncio = time.time() - start_time
print(f"Time taken: {duration_asyncio}")

Time taken: 1.7198710441589355


In [20]:
#Comparison
print(f"Synchronous: {duration_synch}")
print(f"Multithreading: {duration_multithreading}")
print(f"Asyncio: {duration_asyncio}")

Synchronous: 9.95365858078003
Multithreading: 2.4338181018829346
Asyncio: 1.7198710441589355
